# import

In [1]:
import pandas as pd
import openpyxl
import tensorflow as tf
import numpy as np
import os, sys
from memory_profiler import profile

2024-11-15 08:58:53.009385: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 08:58:53.009408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 08:58:53.010278: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 08:58:53.014389: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-15 08:58:53.657050: W tensorflow/compiler/tf2

In [2]:
sys.path.append('/media/jczars/4C22F02A22F01B22/Pollen_classification_view/')
print(sys.path)

from models import get_data, models_train, get_calssifica, utils, reports_build

working_dir="/media/jczars/4C22F02A22F01B22/Pollen_classification_view/"
os.chdir(working_dir)

['/media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels', '/home/jczars/anaconda3/envs/tf/lib/python310.zip', '/home/jczars/anaconda3/envs/tf/lib/python3.10', '/home/jczars/anaconda3/envs/tf/lib/python3.10/lib-dynload', '', '/home/jczars/.local/lib/python3.10/site-packages', '/home/jczars/anaconda3/envs/tf/lib/python3.10/site-packages', '/media/jczars/4C22F02A22F01B22/Pollen_classification_view/']


/home/jczars/anaconda3/envs/tf/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


['/media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels', '/home/jczars/anaconda3/envs/tf/lib/python310.zip', '/home/jczars/anaconda3/envs/tf/lib/python3.10', '/home/jczars/anaconda3/envs/tf/lib/python3.10/lib-dynload', '', '/home/jczars/.local/lib/python3.10/site-packages', '/home/jczars/anaconda3/envs/tf/lib/python3.10/site-packages', '/media/jczars/4C22F02A22F01B22/Pollen_classification_view/', '/media/jczars/4C22F02A22F01B22/$WinREAgent/Pollen_classification_view/']


# Functions

## rec

In [3]:
%load_ext memory_profiler
def rec_id(workbook_path, id_test):
    # Load configuration data from 'Sheet'
    config_data = pd.read_excel(workbook_path, sheet_name="Sheet")
    config = config_data.loc[id_test]
    rec_csv = pd.read_excel(workbook_path, sheet_name="Table")
    fil=rec_csv[rec_csv['id_test'] == id_test]
    tempo_px=len(fil)

    return config, tempo_px

## train_model

In [4]:
%load_ext memory_profiler
def train_model(config, train_data, val_data, time_step):
    """
    Train a model with the given configuration and data. If time_step > 0, it will load a pre-trained model
    and continue training. Otherwise, it will train a new model from scratch.

    Parameters
    ----------
    config : dict
        A dictionary containing the configuration for model training.
    train_data : tuple
        A tuple containing the training data and labels.
    val_data : tuple
        A tuple containing the validation data and labels.
    time_step : int
        The current training step. If time_step > 0, it will load a model from the previous step.

    Returns
    -------
    model_inst : keras.Model
        The trained or reloaded model instance.
    res_train : dict
        A dictionary containing the training history and metrics.
    """
    print('\n[INFO]--> time_step ', time_step)
    
    # Reset model_inst to ensure it starts fresh for each time step
    model_inst = None
    
    # If time_step > 0, try to load the model from a previous step, otherwise train a new model
    if time_step > 0:
        # Build the model path for the previous step
        model_name = f"{config['id_test']}_{config['model']}_bestLoss_{time_step - 1}.keras"
        save_path = os.path.join(config['save_dir'], model_name)
        
        # Load the model from the previous time step
        if os.path.exists(save_path):
            print(f"[INFO]--> Loading model from {save_path}")
            model_inst = tf.keras.models.load_model(save_path)
            
            # Explicitly freeze the layers again
            for layer in model_inst.layers:
                layer.trainable = False  # Freeze all layers
            
            # Optionally unfreeze the last few layers if needed (based on config['freeze'])
            for i, layer in enumerate(model_inst.layers):
                if i >= config['freeze']:
                    layer.trainable = True  # Unfreeze layers after the specified freeze index
            
            print(f"[INFO]--> Model layers frozen up to layer {config['freeze']}")
        else:
            raise ValueError(f"[ERROR]--> Model from time_step {time_step - 1} not found at {save_path}")
    
    # else:
    #     # Instantiate the model from scratch for time_step == 0
    #     print("[INFO]--> Training a new model from scratch...")
    #     model_inst = models_pre.hyper_model_up(config, verbose=1)
    
    # Train the model with the training and validation data
    res_train = models_train.run_train(train_data, val_data, model_inst, config)
    
    # Save the model at the current time step
    model_name = f"{config['id_test']}_{config['model']}_bestLoss_{time_step}.keras"
    save_path = os.path.join(config['save_dir'], model_name)
    
    # Save the trained model
    model_inst.save(save_path)
    print(f"[INFO]--> Model saved at {save_path}")
    
    return model_inst, res_train

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


## build_reports_config

In [5]:
@profile
def build_reports_config(time_step, config, res_pre, model_inst, res_train, verbose=0):
    """
    Generates evaluation reports for a model based on given configurations, test data, and training results.

    Parameters:
    - time_step (int/float): The time step or timestamp associated with the evaluation.
    - config (dict): A dictionary containing model and report configuration parameters, such as image size and batch size.
    - res_pre (dict): Contains preprocessing results, including the test data path and category information.
    - model_inst: The trained model instance.
    - res_train (dict): Contains training results, including training history.
    - verbose (int, optional): Level of verbosity for printing messages. Default is 0 (no output).

    Returns:
    - report_metrics (Any): Generated metrics from the report generation process.

    Function Workflow:
    1. Conditionally prints log messages indicating the start of report generation based on verbosity level.
    2. Loads test data from a specified path.
    3. Prepares the input size for data processing.
    4. Loads test data for evaluation.
    5. Creates necessary directories for report storage.
    6. Configures and generates reports using provided data and model.
    """
    if verbose > 0:
        print('\nReports Generation')
        print(f'\n[INFO]--> Step 1.4 - Evaluation Time Step: {time_step}')
    
    # Load test data
    test_data = pd.read_csv(res_pre['path_test'])
    print("\n[INFO]--> res_pre['path_test']", res_pre['path_test'])
    print('\n[INFO]--> test_data.head()', test_data.head())

    img_size = config['img_size']
    input_size = (img_size, img_size)
    
    if verbose > 0:
        print(f'\n[INFO]--> Input size: {input_size}')
    
    # Load processed test data
    test = get_data.load_data_test(test_data, input_size)
    categories = res_pre['categories']
    
    # Create report saving directory
    save_dir = os.path.join(res_pre['save_dir_train'], 'reports')
    utils.create_folders(save_dir, 0)
    
    # Configure report generation settings
    reports_config = {
        'save_dir': save_dir,
        'time': time_step,
        'batch_size': config['batch_size'],
        'id_test': config['id_test'],
        'model': config['model']
    }
    
    # Generate reports
    history = res_train['history']
    report_metrics = reports_build.reports_gen(test, model_inst, categories, history, reports_config)
    
    return report_metrics

## classification

In [6]:
@profile
def classification(config, res_pre, model, _tempo, verbose=0):
    """
    Classifies unlabeled images and generates pseudo-labels.

    Args:
        config (dict): Model configuration and directory paths.
        res_pre (dict): Previous results, including categories and pseudo-label paths.
        model (torch.nn.Module): Trained model used for predictions.
        _tempo (int): Current iteration of the pseudo-labeling process.
        verbose (int, optional): Verbosity level for printing messages. Default is 0 (no output).

    Returns:
        pd.DataFrame: DataFrame containing pseudo-label predictions, or None if data is unavailable.
    """
    # Define the path for the unlabeled dataset
    unlabels_path = os.path.join(config['path_base'], 'images_unlabels')
    batch_size = config['batch_size']
    categories = res_pre['categories']

    params = {
        'unlabels': unlabels_path,
        'img_size': config['img_size'],
        'batch_size': batch_size,
        'categories': categories
    }

    # Load unlabeled data or read CSV with previous pseudo-labels
    if _tempo == 0:
        if verbose > 0:
            print(f"[DEBUG] Loading unlabeled images from directory: {unlabels_path}")
        unlabels_generator = get_data.load_unlabels(params)
    else:
        unlabels_csv_path = os.path.join(res_pre['pseudo_csv'], f'unlabelSet_T{_tempo}.csv')
        #/media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels/Reports/5_DenseNet201_sem_BI_5/pseudo_csv/unlabelSet_T14.csv
        if verbose > 0:
            print(f"[DEBUG] Loading pseudo-labels from CSV: {unlabels_csv_path}")
        try:
            df_unlabels = pd.read_csv(unlabels_csv_path)
        except FileNotFoundError:
            if verbose > 0:
                print(f"[ERROR] File not found: {unlabels_csv_path}")
            return None

        if len(df_unlabels) > 0:
            if verbose > 0:
                print("[DEBUG] Head of the pseudo-labels DataFrame:")
                print(df_unlabels.head())
            unlabels_generator = get_data.load_data_test(df_unlabels, input_size=(224, 224))
        else:
            if verbose > 0:
                print(f"[WARNING] No data found in CSV {unlabels_csv_path}")
            return None

    # Perform predictions for pseudo-labeling
    if verbose > 0:
        print("[INFO] Performing pseudo-labeling on the unlabeled dataset")
    
    pseudos_df = reports_build.predict_unlabeled_data(
        unlabels_generator, model, batch_size, categories, verbose=verbose
    )
    
    if verbose > 0:
        print(f"[INFO] Total pseudo-labels generated: {len(pseudos_df)}")

    return pseudos_df

## selection

In [7]:
@profile
def selection(pseudos_df, conf, res_pre, _tempo, verbose=0):
    """
    Performs selection of pseudo-labels for training if unlabeled data is available.

    Steps:
    1. Checks if there is any unlabeled data.
    2. Calls the `selec` function to select pseudo-labels based on a confidence threshold.
    3. Returns a dictionary with paths and sizes of datasets if selection is successful.
    4. Returns None if no selection could be made or if there is no unlabeled data.

    Parameters:
        pseudos_df (DataFrame): Unlabeled data to be processed.
        conf (dict): Configuration dictionary with paths and threshold settings.
        res_pre (dict): Contains the path for saving pseudo-labels.
        _tempo (int): Current time or iteration index.
        training_data (Any): Training data used for comparison or updating with pseudo-labels.
        verbose (int, optional): Verbosity level for printing messages. Default is 0 (no output).

    Returns:
        dict or None: Returns a dictionary with new data paths and dataset sizes if successful; 
                      returns None if no selection was made or no unlabeled data.
    """
    if not pseudos_df.empty:
        if verbose > 0:
            print('\n[STEP 2].4 - Selection')

        # Load the training data 
        try:
            training_data = pd.read_csv(res_pre['path_train'])
        except (FileNotFoundError, pd.errors.EmptyDataError) as e:
            raise ValueError(f"Error loading training data: {e}")


        if verbose > 0:
            print(f'Training data provided: {training_data}')

        # Perform pseudo-label selection
        res_sel = get_calssifica.selec(
            conf,
            pseudos_df,
            res_pre['pseudo_csv'], 
            _tempo, 
            training_data,
            conf['limiar']
        )

        if res_sel:
            # Return paths and dataset sizes for further processing
            return {
                '_csv_New_TrainSet': res_sel['_csv_New_TrainSet'],
                'path_test': res_pre['path_test'],
                'save_dir_train': conf.get('path_model', ''),  # Assuming model save path in config
                'pseudo_csv': res_pre['pseudo_csv'],
                'ini': res_sel['ini'],
                'select': res_sel['select'],
                'rest': res_sel['rest'],
                'train': res_sel['train'],
                'new_train': res_sel['new_train']
            }
        else:
            # No valid pseudo-labels selected
            if verbose > 0:
                print("[INFO] No valid pseudo-labels were selected.")
            return None
    else:
        # No unlabeled data to process
        if verbose > 0:
            print("[INFO] No unlabeled data available for processing.")
        return None

In [8]:
@profile
def rel_data(time_step, report_metrics, res_train, res_sel, workbook_path, config_index, verbose=0):
    """
    Saves data into an Excel workbook for reporting purposes.

    Parameters:
        time_step (str/int): Current time step or identifier for data logging.
        report_metrics (dict): Metrics from the report generation process.
        res_train (dict): Training result data, including timing and accuracy metrics.
        res_sel (dict): Selection result data, containing training set sizes and other statistics.
        workbook_path (str): Path to the Excel workbook.
        config_index (str/int): Identifier for the configuration used.
        verbose (int, optional): Verbosity level for printing messages. Default is 0 (no output).
    """
    if verbose > 0:
        print("\n[INFO] Workbook name:", workbook_path)
    
    try:
        workbook = openpyxl.load_workbook(workbook_path)
        if verbose > 0:
            print("Sheets in workbook:", workbook.sheetnames)
    except FileNotFoundError:
        if verbose > 0:
            print("[ERROR] Workbook not found, creating a new one.")
        workbook = openpyxl.Workbook()

    sheet_name = 'Table'
    
    # Check if the sheet already exists
    if sheet_name in workbook.sheetnames:
        if verbose > 0:
            print(f'Sheet "{sheet_name}" exists.')
        Met_page = workbook[sheet_name]  # Access the existing sheet
    else:
        if verbose > 0:
            print(f'Creating new sheet: "{sheet_name}".')
        Met_page = workbook.create_sheet(sheet_name)  # Create a new sheet
        if verbose > 0:
            print('[INFO] -rel_data- Saving test header.')
        cols_exe = ['Tempo', 'test_loss', 'test_accuracy', 'precision', 'recall', 'fscore', 
                    'kappa', 'str_time', 'end_time', 'delay', 'best_epoch', 
                    'ini', 'select', 'rest', 'train', 'new_train', 'id_test']
        Met_page.append(cols_exe)  # Add header row with column names
    
    # Append data to the sheet
    data = [
        str(time_step),
        report_metrics.get('test_loss', ''),
        report_metrics.get('test_accuracy', ''),
        report_metrics.get('precision', ''),
        report_metrics.get('recall', ''),
        report_metrics.get('fscore', ''),
        report_metrics.get('kappa', ''),
        res_train.get('start_time', ''),
        res_train.get('end_time', ''),
        res_train.get('duration', ''),
        res_train.get('best_epoch', ''),
        res_sel.get('ini', ''),
        res_sel.get('select', ''),
        res_sel.get('rest', ''),
        res_sel.get('train', ''),
        res_sel.get('new_train', ''),
        config_index
    ]
    Met_page.append(data)
    
    # Save the workbook
    workbook.save(workbook_path)
    
    if verbose > 0:
        print("Data saved successfully. Sheets available:", workbook.sheetnames)

# Main

In [9]:
def test1(workbook_path, id_test, verbose=1):
    has_unlabeled_data = True
    print('\n[STEP] Recovery phase')
    #%memit  
    config, time_step = rec_id(workbook_path, id_test)
    print("\nconfig", config)
    print("\ntempo_px", time_step) 

    while has_unlabeled_data: 
        print(f'*'*30)               
        print('\n[STEP] Train phase')

        confi_load={
            'aug': config['aug'],
            'img_size': config['img_size'],
        }

        # recurarar csv_NewtainSet14
        _pseudo_csv=f'{working_dir}/0_pseudo_labels/Reports/{id_test}_{config["model"]}_{config["aug"]}_{config["base"]}/pseudo_csv/'
        _csv_New_TrainSet = os.path.join(_pseudo_csv, f'trainSet_T{time_step}.csv')
        print(_csv_New_TrainSet)

        save_dir = f'{working_dir}/0_pseudo_labels/Reports/{id_test}_{config["model"]}_{config["aug"]}_{config["base"]}/models/'
        
        config_train={
        'id_test': id_test,
        'model': config['model'],
        'save_dir': save_dir,
        'freeze': config['freeze'],
        'batch_size': config['batch_size'],
        'epochs': config['epochs'],
        }

        train, val = get_data.reload_data_train(confi_load, _csv_New_TrainSet)

        model_train, res_train = train_model(config_train, train, val, time_step)

        # Label directory path
        labels_dir = os.path.join(config['path_base'], "labels")
        categories = sorted(os.listdir(labels_dir))

        class_config={'path_base': config['path_base'],
                    'batch_size': config['batch_size'],
                    'img_size': config['img_size'],
                    'categories': categories,
                    'pseudo_csv': _pseudo_csv
        }
        #/media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels/Reports/BI_5_testSet.csv
        path_test= f"{working_dir}/0_pseudo_labels/Reports/{config['base']}_testSet.csv"
        train_path= f"{working_dir}/0_pseudo_labels/Reports/{config['base']}_trainSet.csv"
        save_dir_train= f'{working_dir}/0_pseudo_labels/Reports/{id_test}_{config["model"]}_{config["aug"]}_{config["base"]}'
        num_labels = len(categories)
        print(path_test)
        res_pre = {
            'path_train': train_path,
            'path_test': path_test,
            'save_dir_train': save_dir_train,
            'pseudo_csv': _pseudo_csv,
            'size_of_labels': num_labels,
            'categories': categories,
        }

        report_metrics = build_reports_config(time_step, config, res_pre, model_train, res_train)
        
        print('\n[STEP] Classification phase')
        pseudos_df = classification(class_config, res_pre, model_train, time_step)  

        print('\n[STEP] Selection phase')
        res_sel = selection(pseudos_df, config, res_pre, time_step)

        if res_sel is None:
            if verbose > 0:
                print("[INFO] No more unlabeled data to process.")
            break
        print('\n[STEP] save reports')
        rel_data(time_step, report_metrics, res_train, res_sel, workbook_path, time_step)
        
        time_step += 1
        print(f'*'*30)   
        print('\n[STEP] Next step', time_step)

In [ ]:
if __name__ == "__main__":
    workbook_path = "/media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels/Reports/config_pseudo_label_pre.xlsx"
    start_index = 5

    test1(workbook_path, start_index)



[STEP] Recovery phase

config id_test                      5
base                      BI_5
path_base              BD/BI_5
aug                        sem
img_size                   224
batch_size                   4
epochs                       1
alpha                  0.00001
model              DenseNet201
last_activation        softmax
activation                relu
dense_size                1024
dropout                    0.3
freeze                       0
split_valid                0.2
optimizer              RMSprop
learning_rate           0.0001
type_model            imagenet
limiar                   0.995
Name: 5, dtype: object

tempo_px 14

[STEP] Train phase
/media/jczars/4C22F02A22F01B22/Pollen_classification_view//0_pseudo_labels/Reports/5_DenseNet201_sem_BI_5/pseudo_csv/trainSet_T14.csv
training_data_path  /media/jczars/4C22F02A22F01B22/Pollen_classification_view//0_pseudo_labels/Reports/5_DenseNet201_sem_BI_5/pseudo_csv/trainSet_T14.csv

 ######## Data Generator ##########

2024-11-15 08:58:56.487352: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-15 08:58:56.513004: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-15 08:58:56.513135: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

[INFO]--> Model layers frozen up to layer 0
Batch size: 4
Training start time: 2024-11-15 09:00:52

 2024-11-15 09:00:52


2024-11-15 09:01:18.135281: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-11-15 09:01:29.039939: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f233c5d2630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-15 09:01:29.039954: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-11-15 09:01:29.043397: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731672089.088660   60797 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


91/91 [==============================] - 135s 675ms/step - loss: 0.0045 - accuracy: 0.9983 - val_loss: 0.4322 - val_accuracy: 0.9349
Training duration: 0:02:15
Best epoch: 1 with validation loss: 0.4322
[INFO]--> Model saved at /media/jczars/4C22F02A22F01B22/Pollen_classification_view//0_pseudo_labels/Reports/5_DenseNet201_sem_BI_5/models/5_DenseNet201_bestLoss_14.keras
/media/jczars/4C22F02A22F01B22/Pollen_classification_view//0_pseudo_labels/Reports/BI_5_testSet.csv
ERROR: Could not find file /tmp/ipykernel_60705/1460078313.py

[INFO]--> res_pre['path_test'] /media/jczars/4C22F02A22F01B22/Pollen_classification_view//0_pseudo_labels/Reports/BI_5_testSet.csv

[INFO]--> test_data.head()                                                 file            labels
0  BD/BI_5/labels/polar_circular/9.Origanum (46).png    polar_circular
1  BD/BI_5/labels/polar_triangular/12.Calicotome ...  polar_triangular
2  BD/BI_5/labels/polar_circular/9.Origanum (11).png    polar_circular
3  BD/BI_5/labels/pol

/media/jczars/4C22F02A22F01B22/Pollen_classification_view/models/reports_build.py:315: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(ax.get_yticklabels(), fontsize=10, rotation=0)



[STEP] Classification phase
ERROR: Could not find file /tmp/ipykernel_60705/1881900816.py
Found 66 validated image filenames belonging to 5 classes.
Predicting unlabeled data... 66
17/17 [==============================] - 1s 35ms/step

[STEP] Selection phase
ERROR: Could not find file /tmp/ipykernel_60705/2036738533.py
                                                file             labels  \
0  BD/BI_5/images_unlabels/unlabels/1.Thymbra (11...   polar_triangular   
1  BD/BI_5/images_unlabels/unlabels/1.Thymbra (13...   polar_triangular   
2  BD/BI_5/images_unlabels/unlabels/1.Thymbra (14...  polar_tricircular   
3  BD/BI_5/images_unlabels/unlabels/1.Thymbra (28...     polar_circular   
4  BD/BI_5/images_unlabels/unlabels/1.Thymbra (51...   polar_triangular   

   confidence  
0    0.355792  
1    0.895780  
2    0.518683  
3    0.999879  
4    0.827282  
Initial Data Preview:                                                 file             labels  \
0  BD/BI_5/images_unlabels/unlabel

/media/jczars/4C22F02A22F01B22/Pollen_classification_view/models/reports_build.py:315: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(ax.get_yticklabels(), fontsize=10, rotation=0)



[STEP] Classification phase
ERROR: Could not find file /tmp/ipykernel_60705/1881900816.py
Found 39 validated image filenames belonging to 6 classes.
Predicting unlabeled data... 39
10/10 [==============================] - 1s 60ms/step

[STEP] Selection phase
ERROR: Could not find file /tmp/ipykernel_60705/2036738533.py
                                                file               labels  \
0  BD/BI_5/images_unlabels/unlabels/1.Thymbra (11...  equatorial_circular   
1  BD/BI_5/images_unlabels/unlabels/1.Thymbra (13...  equatorial_circular   
2  BD/BI_5/images_unlabels/unlabels/1.Thymbra (14...       polar_circular   
3  BD/BI_5/images_unlabels/unlabels/1.Thymbra (51...       polar_circular   
4  BD/BI_5/images_unlabels/unlabels/1.Thymbra (86...       polar_circular   

   confidence  
0    0.715262  
1    0.999151  
2    0.928889  
3    0.700651  
4    0.986535  
Initial Data Preview:                                                 file               labels  \
0  BD/BI_5/images_un